In [2]:
__author__='Xia Wang'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
s = json.load(open(os.getenv('PUI2015')+'/plotstyle.json'))
plt.rcParams.update(s)

%matplotlib inline

/Users/eliseowang/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:855: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Task 1

In [ ]:
data = np.load(os.getenv('PUI2015')+'/HW6/MTA_Fare.npy')

In [ ]:
print data.shape
names=np.array(['10t', '14d', '1d', '2t', '30d', '7d', 'ada', 'afas', 'exp','ez',
            'ff','mc','mr','month','rfm','rr','sen','spec','tcmc','t',
            'umlim','xbus','stud'])   

In [ ]:
data = np.where(data != -1, data, np.nan)

In [ ]:
# add up all the stations
by_ridetype_sum = np.nansum(data, axis=0)
# add up all ride types
by_station_sum = np.nansum(data, axis=1)

In [ ]:
station1 = pd.DataFrame(by_station_sum.T)
station1.index = pd.date_range('5/21/2010',periods=194,freq='W-FRI')

In [ ]:
fig1 = plt.figure(figsize=(10,6))
fig1.add_subplot(111)
plt.plot(station1.index, np.log10(station1))
plt.title('swipe No. by station (sum)',fontsize=20)
plt.xticks(rotation=270)
plt.xlabel('Week No.', fontsize=15)
plt.ylabel('Swipe No., log10 scale', fontsize=15)
plt.grid()
plt.show

In [ ]:
ridetype1 = pd.DataFrame(by_ridetype_sum.T)
ridetype1.columns = ['10t', '14d', '1d', '2t', '30d', '7d', 'ada', 'afas', 'exp','ez',
                    'ff','mc','mr','month','rfm','rr','sen','spec','tcmc','t','umlim',
                    'xbus','stud']
ridetype1.index = pd.date_range('5/21/2010',periods=194,freq='W-FRI')

In [ ]:
fig1 = plt.figure(figsize=(10,6))
fig1.add_subplot(111)
plt.plot(ridetype1.index, np.log10(ridetype1))
plt.title('swipe No. by ridetype (sum)',fontsize=20)
plt.xticks(rotation=270)
plt.xlabel('Week No.', fontsize=15)
plt.ylabel('Swipe No. log10 scale', fontsize=15)
plt.grid()
plt.show

In [ ]:
by_ridetype_avg = np.nanmean(data, axis=0)
by_station_avg = np.nanmean(data, axis=1)

In [ ]:
# print by_ridetype_avg.shape
# print by_station_avg.shape

In [ ]:
ridetype = pd.DataFrame(by_ridetype_avg.T)

In [ ]:
ridetype.columns = ['10t', '14d', '1d', '2t', '30d', '7d', 'ada', 'afas', 'exp','ez',
                    'ff','mc','mr','month','rfm','rr','sen','spec','tcmc','t','umlim',
                    'xbus','stud']

In [ ]:
ridetype.index = pd.date_range('5/21/2010',periods=194,freq='W-FRI')

In [ ]:
station = pd.DataFrame(by_station_avg.T)

In [ ]:
station.index = pd.date_range('5/21/2010',periods=194,freq='W-FRI')

In [ ]:
fig1 = plt.figure(figsize=(10,6))
fig1.add_subplot(111)
plt.plot(station.index, np.log10(station))
plt.title('swipe No. by station (average)', fontsize=20)
plt.xticks(rotation=270)
plt.xlabel('Week No.', fontsize=15)
plt.ylabel('Swipe No. log10 scale', fontsize=15)
plt.grid()
plt.show

In [ ]:
fig1 = plt.figure(figsize=(10,6))
fig1.add_subplot(111)
plt.plot(ridetype.index, np.log10(ridetype))
plt.title('swipe No. by ridetype (average)', fontsize=20)
plt.xticks(rotation=90)
plt.xlabel('Week No.', fontsize=15)
plt.ylabel('Swipe No. log10 scale', fontsize=15)
plt.grid()
plt.show

## By simply looking at the plots (both by station and by ridetype), we can detect a huge drop in the winter of 2012. I'll detect the event as an outlier using 3 standard deviations from mean as the critical value. And I'll only apply it to the swipe numbers by ridetype (average). The same code can be applied to other datasets.

In [ ]:
ridetype_num = pd.DataFrame(by_ridetype_avg.T).convert_objects(convert_numeric=True)

In [ ]:
ridetype_num.loc['sigma'] = ridetype_num.std()
ridetype_num.loc['mean'] = ridetype_num.mean()
ridetype_num.loc['critical min'] = ridetype_num.loc['mean'] - 3.0*ridetype_num.loc['sigma']
ridetype_num.loc['critical max'] = ridetype_num.loc['mean'] + 3.0*ridetype_num.loc['sigma']

In [ ]:
# calculate if any value in the dataframe is an outliers
eventsmin = np.where(ridetype_num < ridetype_num.loc['critical min'])
eventsmax = np.where(ridetype_num > ridetype_num.loc['critical max'])[0]

In [ ]:
print eventsmin

In [ ]:
events = np.array(eventsmin[0])
np.unique(events)

## According to the calculation, week 126, 173 189 and 194 have outliers. As we have seen previously in the plots, in week 126 the plot shows a sharp drop. Week 173, 189 are not very obvious on the plots though. As for week 126, the sharp drop was probably caused by Hurricane Sandy when the infrustructure was damaged and the use of metro dropped significantly.

## Task 2

In [ ]:
from scipy import stats
import statsmodels.api as sm

In [ ]:
# I've tried different window numbers from 1 to 20 and I think 10 is a good fit.
pd.rolling_mean(ridetype, 10).plot(figsize=(10,6), 
                                   title='Rolling mean of ridetype', fontsize=20)
plt.legend(loc=8)
for x in ridetype.ix['2010-08-06']:
    plt.axhline(x, color='k',linestyle='--')

## 10t shows a growing trend, unlim is almost stable, 7d shows an even more obvious growing trend, and ada type dropped a little. The others are more difficult to detect by eyes. The whole result is shown below

In [ ]:
ls1 = pd.rolling_mean(ridetype, 10).dropna().head(10)
ls2 = pd.rolling_mean(ridetype, 10).dropna().tail(10)

In [ ]:
# I calculated the rolling mean for the first and last 10 weeks,
# the one by the other, and took the average
ls1.index = np.arange(10)
ls2.index = np.arange(10)
ratio = ls1/ls2
ratio.mean()

## The rolling std is shown below

In [ ]:
pd.rolling_std(ridetype, 12).plot(figsize=(10,6))
plt.legend(loc=9)

## Task 3

In [ ]:
f = np.abs(np.fft.rfft(by_station_sum))
f.shape

In [ ]:
N =194
ffreq = np.fft.rfftfreq(N, 1.0)
ffreq.shape

In [ ]:
station1 = station1.T.astype(float)

In [ ]:
fig = plt.figure(figsize=(10,300))
for i in range(600):
    ax = fig.add_subplot(300,2,i)
    f = np.abs(np.fft.rfft(station1.loc[i]))
    ax.plot(np.fft.rfftfreq(N, 1.0)[1:], (f[1:]), 'o', ms=5)
    ax.set_title('%i' %(i-1))
    ax.plot(np.fft.rfftfreq(N, 1.0)[1:], (f[1:]), '-')
    plt.axvline(1.0/52)

In [ ]:
fig = plt.figure(figsize=(10,300))
f = np.abs(np.fft.rfft(station1.loc[i]))
ax.plot(np.fft.rfftfreq(N, 1.0)[1:], (f[1:]), 'o', ms=5)
ax.plot(np.fft.rfftfreq(N, 1.0)[1:], (f[1:]), '-')
plt.axvline(1.0/52)

In [ ]:
# fig=plt.figure(figsize=(15,5))
# # print (1.0/np.fft.rfftfreq(N, 1.0)[0:2])
# # print (1.0/np.fft.rfftfreq(N, 1.0)[-3:-1])
# X = np.fft.rfftfreq(N, 1.0)
# plt.plot(X, f.T, 'o', ms=20)
# plt.plot(X, f.T, '-', ms=20)
# #plt.set_xticklabels([ "%s"%(1/f) for f in ax.get_xticks()], fontsize=20)
# plt.xlabel("period (week)", fontsize=20)
# plt.ylabel("Power", fontsize=20)
# plt.title("Trash pickup power spectrum", fontsize=20)